# Episode 21 — Execute on Target Hardware

In this notebook you will run optimized ISA circuits on **IBM Quantum** hardware using **Qiskit Runtime primitives**. You will:

- Initialize the Runtime service and pick a backend
- Build a parametric circuit and a Pauli observable
- Transpile to the backend's instruction set (ISA)
- Execute with **EstimatorV2** (expectation values)
- Execute with **SamplerV2** (shot-based bitstrings)


## 1) Initialize account and select a backend
If you haven't saved your IBM Quantum credentials locally yet, follow the official setup docs first. Then select an operational backend.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Initialize the Runtime service (uses saved credentials if present)
service = QiskitRuntimeService()

# Pick a real device (adjust filters as needed)
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
print(f"Selected backend: {backend.name}")

## 2) Build a circuit + observable for Estimator
We'll create a QAOA-style ansatz and a ZZ observable over the device connectivity graph. The parameter values are placeholders for demonstration.

In [ ]:
from qiskit.circuit.library import qaoa_ansatz
from qiskit.quantum_info import SparsePauliOp

# Build a ZZ observable using the device's coupling map
edges = [tuple(edge) for edge in backend.coupling_map.get_edges()]
observable = SparsePauliOp.from_sparse_list(
    [("ZZ", [i, j], 0.5) for i, j in edges],
    num_qubits=backend.num_qubits,
)

# Create a QAOA-like ansatz from the observable
circuit = qaoa_ansatz(observable, reps=2)
param_values = [0.1, 0.2, 0.3, 0.4]  # demo values for the ansatz parameters

print("Circuit qubits:", circuit.num_qubits)
print("Params:", len(circuit.parameters))

## 3) Transpile to ISA (device-compatible)
This step rewrites the circuit to match the backend's basis gates and connectivity. We'll also map the observable to the transpiled circuit layout.

In [ ]:
from qiskit.transpiler import generate_preset_pass_manager

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

print("ISA circuit op counts:", isa_circuit.count_ops())
isa_circuit.draw("mpl")

## 4) Execute with EstimatorV2 (expectation values)
Estimator computes expectation values of observables with respect to states prepared by your circuit.

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator

estimator = Estimator(mode=backend)
job = estimator.run([(isa_circuit, isa_observable, param_values)])
print("Estimator Job ID:", job.job_id())

result = job.result()
print("Expectation value:", result[0].data.evs)
print("Metadata:", result[0].metadata)

## 5) Execute with SamplerV2 (bitstrings)
Sampler returns per-shot measurement outcomes. We'll create a simple variational circuit, measure all qubits, transpile, then run with Sampler.

In [ ]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit_ibm_runtime import SamplerV2 as Sampler

# Build a measured circuit for sampling
sampler_circ = EfficientSU2(backend.num_qubits, entanglement="linear")
sampler_circ.measure_all()

# Random parameter vector for the variational layers
param_vec = np.random.rand(sampler_circ.num_parameters)

# Transpile to ISA
isa_sampler_circ = pm.run(sampler_circ)

sampler = Sampler(mode=backend)
job_s = sampler.run([(isa_sampler_circ, param_vec)])
print("Sampler Job ID:", job_s.job_id())

res_s = job_s.result()
bitstrings = res_s[0].data.meas.get_bitstrings()
print("First 10 bitstrings:", bitstrings[:10])

## 6) (Optional) Tune Runtime options
You can set options like resilience level, twirling, or dynamical decoupling. Uncomment and adapt as needed.

In [ ]:
# from qiskit_ibm_runtime import EstimatorV2 as Estimator
# options = {
#     "resilience_level": 1,                # 0, 1, or 2
#     "twirling": {"enable_measure": True},
#     "execution": {"init_qubits": True},
# }
# est_with_opts = Estimator(mode=backend, options=options)
# job_opt = est_with_opts.run([(isa_circuit, isa_observable, param_values)])
# res_opt = job_opt.result()
# print("EV (with options):", res_opt[0].data.evs)

## Additional information

**Created by:** Ricard Santiago Raigada García

**Version:** 1.0.0